In [1]:
%%capture
!pip install transformers

In [2]:
%%capture
!pip install datasets

In [3]:
import transformers

In [4]:
model_checkpoint = "xlm-roberta-base"

In [5]:
from transformers import  AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
isinstance(tokenizer, transformers.PreTrainedTokenizerFast)
tokenizer.model_max_length

512

In [6]:
from datasets import load_dataset, load_metric

toke_tagging_dataset = load_dataset('drive/MyDrive/Colab Notebooks/load_dataset.py')

['/content/drive/MyDrive/Colab Notebooks/corpus/big_dataset.txt', '/content/drive/MyDrive/Colab Notebooks/corpus/gold_corpus_test.txt']


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

['/content/drive/MyDrive/Colab Notebooks/corpus/big_dataset.txt', '/content/drive/MyDrive/Colab Notebooks/corpus/gold_corpus_test.txt']


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset load_dataset downloaded and prepared to /root/.cache/huggingface/datasets/load_dataset/token_tagging/1.0.0/7c799514eb3c22c21d4f96973757a13e4b85d4337ef06fce5d400a82aadcc856. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
label_list = toke_tagging_dataset['train'].features['labels'].feature.names

In [8]:
model = transformers.AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=3)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

In [9]:
model_name = model_checkpoint.split("/")[-1]
task = 'error_detection'
batch_size = 16

args = transformers.TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit = 1,
)

In [10]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [11]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 2.0 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=3aaddeb9ec7bb6dd5b8a13d69c99963220c5293d35e10348f74cdcf8f2562a3f
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [12]:
metric = load_metric("seqeval")

In [13]:
toke_tagging_dataset['train'].features

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=ClassLabel(num_classes=3, names=['ok', 'dif', 'del'], id=None), length=-1, id=None)}

In [14]:
example = toke_tagging_dataset["train"][4]

In [15]:
labels = [label_list[i] for i in example['labels']]
metric.compute(predictions=[labels], references=[labels])

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ok seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: dif seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: del seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'el': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4},
 'if': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 7},
 'k': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 9},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [16]:
toke_tagging_dataset['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 833571
})

In [17]:
toke_tagging_dataset['validation']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1519
})

In [18]:
ok = 0
dif = 0
dell = 0

for tags in  toke_tagging_dataset['train'][:100000]['labels']:
  for tag in tags:
    if tag == 0:
      ok +=1
    if tag == 1:
      dif +=1
    if tag == 2:
      dell += 1

total = (ok + dif + dell)
print(total)
print(ok)
print(dif)
print(dell)

weights = [1, 7, 7.9]

3939584
3098914
448322
392348


In [19]:
print(ok/ok)
print(1/(dif/ok)) #6.5
print(1/(dell/ok)) #7.5

1.0
6.9122505699028824
7.8983810290864245


In [20]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [21]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import torch
from torch import nn
from transformers import Trainer


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([7, 7, 1.0], device='cuda'))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [23]:
trainer = Trainer(
    model,
    args,
    train_dataset=toke_tagging_dataset["train"],
    eval_dataset=toke_tagging_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [24]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-f4d36d4f-b059-9acb-6a1f-9d748b4160a8)


In [ ]:
trainer.train()
#model_loaded = AutoModelForSequenceClassification.from_pretrained("Trained model")

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 833571
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 260495


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.172800,0.559913,0.353931,0.233314,0.281236,0.915083
2,0.146100,0.585062,0.371366,0.227194,0.281917,0.919668


Saving model checkpoint to xlm-roberta-base-finetuned-error_detection/checkpoint-500
Configuration saved in xlm-roberta-base-finetuned-error_detection/checkpoint-500/config.json
Model weights saved in xlm-roberta-base-finetuned-error_detection/checkpoint-500/pytorch_model.bin
tokenizer config file saved in xlm-roberta-base-finetuned-error_detection/checkpoint-500/tokenizer_config.json
Special tokens file saved in xlm-roberta-base-finetuned-error_detection/checkpoint-500/special_tokens_map.json
Saving model checkpoint to xlm-roberta-base-finetuned-error_detection/checkpoint-1000
Configuration saved in xlm-roberta-base-finetuned-error_detection/checkpoint-1000/config.json
Model weights saved in xlm-roberta-base-finetuned-error_detection/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in xlm-roberta-base-finetuned-error_detection/checkpoint-1000/tokenizer_config.json
Special tokens file saved in xlm-roberta-base-finetuned-error_detection/checkpoint-1000/special_tokens_map.js

In [ ]:
trainer.save_model("Trained model 800k")